In [1]:
exec(open("./autoencoderhelperfunctions.py").read())
total_df = pd.read_pickle('/users/madke/documents/060519qabegen.pkl')

C:\Users\madke\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
n3 = pd.read_csv('./102420_gazetvecs.csv', index_col = 0)
n4 = n3[~n3.text.str.contains('і')].copy()
n5 = n4[~n4.text.str.contains('журналист')].copy()

In [3]:
n5 = n5.dropna()
n5 = n5.reset_index(drop = True)
R1 = n5.drop(['entry_id','text'], axis = 1)
R1 = R1.dropna()
R1 = R1.fillna(0)
x_train = R1.values

In [9]:
np.random.seed(0)
tf.compat.v1.random.set_random_seed(0)

activ_h = 'relu'
KI_h = 'he_uniform' 

L1 = 75
L2 = 45
L3 = 9
L4 = 3

v_dim = 100
activ_v = 'tanh'
learn = 0.002
opt = tf.optimizers.Adam(lr= learn, amsgrad = True)
loss_fctn = 'cosine_similarity'
en_name = str(L1) + '_' + str(L2) + '_' + str(L3) 

model = Sequential()
#encoder
model.add(Dense(L1, input_dim=v_dim, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(L2, input_dim=L1, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(L3, input_dim=L2, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(L4, input_dim=L3, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(L4, activation=activ_v, name = 'encoded'))

#decoder
model.add(Dense(L3, input_dim=L4, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(L2, input_dim=L3, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(L1, input_dim=L2, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(v_dim, input_dim=L1, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(v_dim, activation=activ_v, name = 'ypred'))


model.compile(loss=loss_fctn, optimizer=opt, metrics=['acc', 'mae', 'msle', 'mse'])

ran for 10000 epochs found peak in val loss and accuracy around 250 epochs (0.923, 0.50)  

In [10]:
history = model.fit(x_train, x_train,
          batch_size=100,
          epochs=500, verbose = 1, validation_split = 0.2)

xprednp = layer_extract('ypred', model, x_train)
xtrainnp = np.array(x_train)
accs = get_cos(xtrainnp, xprednp)
print('Median exp:' + str(np.median(accs)))

np.random.shuffle(xprednp)
np.random.shuffle(xtrainnp)
accsR = get_cos(xprednp, xtrainnp)
print('Median random:' + str(np.median(accsR)))

Train on 21486 samples, validate on 5372 samples
Epoch 1/500
21486/21486 [==============================] - 1s 41us/step - loss: -0.8875 - acc: 0.4644 - mae: 0.0358 - msle: 7.4606e-04 - mse: 0.0024 - val_loss: -0.9057 - val_acc: 0.5719 - val_mae: 0.0328 - val_msle: 5.7189e-04 - val_mse: 0.0018
Epoch 2/500
21486/21486 [==============================] - 1s 30us/step - loss: -0.9199 - acc: 0.4877 - mae: 0.0306 - msle: 5.0504e-04 - mse: 0.0016 - val_loss: -0.9079 - val_acc: 0.5707 - val_mae: 0.0330 - val_msle: 5.7884e-04 - val_mse: 0.0019
Epoch 3/500
21486/21486 [==============================] - 1s 31us/step - loss: -0.9225 - acc: 0.4879 - mae: 0.0331 - msle: 5.7883e-04 - mse: 0.0018 - val_loss: -0.9142 - val_acc: 0.5719 - val_mae: 0.0395 - val_msle: 7.7966e-04 - val_mse: 0.0027
Epoch 4/500
21486/21486 [==============================] - 1s 30us/step - loss: -0.9272 - acc: 0.4968 - mae: 0.0385 - msle: 7.5573e-04 - mse: 0.0025 - val_loss: -0.9148 - val_acc: 0.5672 - val_mae: 0.0435 - val_ms

Epoch 35/500
21486/21486 [==============================] - 1s 26us/step - loss: -0.9340 - acc: 0.5299 - mae: 0.1339 - msle: 0.0071 - mse: 0.0290 - val_loss: -0.9214 - val_acc: 0.5897 - val_mae: 0.1360 - val_msle: 0.0073 - val_mse: 0.0301
Epoch 36/500
21486/21486 [==============================] - 1s 29us/step - loss: -0.9341 - acc: 0.5303 - mae: 0.1369 - msle: 0.0074 - mse: 0.0304 - val_loss: -0.9235 - val_acc: 0.5901 - val_mae: 0.1373 - val_msle: 0.0074 - val_mse: 0.0302
Epoch 37/500
21486/21486 [==============================] - 1s 29us/step - loss: -0.9345 - acc: 0.5310 - mae: 0.1391 - msle: 0.0076 - mse: 0.0313 - val_loss: -0.9225 - val_acc: 0.5974 - val_mae: 0.1414 - val_msle: 0.0079 - val_mse: 0.0326
Epoch 38/500
21486/21486 [==============================] - 1s 26us/step - loss: -0.9350 - acc: 0.5368 - mae: 0.1389 - msle: 0.0076 - mse: 0.0311 - val_loss: -0.9235 - val_acc: 0.5916 - val_mae: 0.1411 - val_msle: 0.0078 - val_mse: 0.0320
Epoch 39/500
21486/21486 [==================

21486/21486 [==============================] - 1s 32us/step - loss: -0.9361 - acc: 0.5376 - mae: 0.1517 - msle: 0.0088 - mse: 0.0370 - val_loss: -0.9296 - val_acc: 0.5944 - val_mae: 0.1487 - val_msle: 0.0085 - val_mse: 0.0354
Epoch 70/500
21486/21486 [==============================] - 1s 31us/step - loss: -0.9354 - acc: 0.5346 - mae: 0.1519 - msle: 0.0089 - mse: 0.0373 - val_loss: -0.9283 - val_acc: 0.5936 - val_mae: 0.1581 - val_msle: 0.0095 - val_mse: 0.0405
Epoch 71/500
21486/21486 [==============================] - 1s 29us/step - loss: -0.9361 - acc: 0.5395 - mae: 0.1512 - msle: 0.0088 - mse: 0.0369 - val_loss: -0.9315 - val_acc: 0.5905 - val_mae: 0.1483 - val_msle: 0.0085 - val_mse: 0.0352
Epoch 72/500
21486/21486 [==============================] - 1s 28us/step - loss: -0.9361 - acc: 0.5421 - mae: 0.1466 - msle: 0.0083 - mse: 0.0347 - val_loss: -0.9318 - val_acc: 0.5974 - val_mae: 0.1494 - val_msle: 0.0086 - val_mse: 0.0354
Epoch 73/500
21486/21486 [==============================]

21486/21486 [==============================] - 1s 30us/step - loss: -0.9360 - acc: 0.5371 - mae: 0.1623 - msle: 0.0099 - mse: 0.0420 - val_loss: -0.9343 - val_acc: 0.5879 - val_mae: 0.1615 - val_msle: 0.0099 - val_mse: 0.0419
Epoch 104/500
21486/21486 [==============================] - 1s 28us/step - loss: -0.9362 - acc: 0.5412 - mae: 0.1612 - msle: 0.0098 - mse: 0.0415 - val_loss: -0.9331 - val_acc: 0.5957 - val_mae: 0.1643 - val_msle: 0.0101 - val_mse: 0.0433
Epoch 105/500
21486/21486 [==============================] - 1s 29us/step - loss: -0.9367 - acc: 0.5404 - mae: 0.1630 - msle: 0.0100 - mse: 0.0425 - val_loss: -0.9326 - val_acc: 0.5975 - val_mae: 0.1595 - val_msle: 0.0098 - val_mse: 0.0409
Epoch 106/500
21486/21486 [==============================] - 1s 28us/step - loss: -0.9367 - acc: 0.5390 - mae: 0.1635 - msle: 0.0101 - mse: 0.0428 - val_loss: -0.9333 - val_acc: 0.5944 - val_mae: 0.1622 - val_msle: 0.0100 - val_mse: 0.0425
Epoch 107/500
21486/21486 [===========================

21486/21486 [==============================] - 1s 40us/step - loss: -0.9365 - acc: 0.5415 - mae: 0.1562 - msle: 0.0093 - mse: 0.0391 - val_loss: -0.9315 - val_acc: 0.5880 - val_mae: 0.1619 - val_msle: 0.0099 - val_mse: 0.0418
Epoch 138/500
21486/21486 [==============================] - 1s 49us/step - loss: -0.9367 - acc: 0.5408 - mae: 0.1565 - msle: 0.0094 - mse: 0.0393 - val_loss: -0.9358 - val_acc: 0.5948 - val_mae: 0.1562 - val_msle: 0.0093 - val_mse: 0.0392
Epoch 139/500
21486/21486 [==============================] - 1s 32us/step - loss: -0.9359 - acc: 0.5403 - mae: 0.1577 - msle: 0.0095 - mse: 0.0398 - val_loss: -0.9321 - val_acc: 0.6009 - val_mae: 0.1589 - val_msle: 0.0096 - val_mse: 0.0407
Epoch 140/500
21486/21486 [==============================] - 1s 33us/step - loss: -0.9364 - acc: 0.5397 - mae: 0.1631 - msle: 0.0100 - mse: 0.0425 - val_loss: -0.9350 - val_acc: 0.5940 - val_mae: 0.1617 - val_msle: 0.0099 - val_mse: 0.0419
Epoch 141/500
21486/21486 [===========================

...

21486/21486 [==============================] - 1s 28us/step - loss: -0.9385 - acc: 0.5509 - mae: 0.1773 - msle: 0.0116 - mse: 0.0498 - val_loss: -0.9383 - val_acc: 0.6082 - val_mae: 0.1783 - val_msle: 0.0117 - val_mse: 0.0506
Epoch 341/500
21486/21486 [==============================] - 1s 29us/step - loss: -0.9389 - acc: 0.5545 - mae: 0.1764 - msle: 0.0115 - mse: 0.0494 - val_loss: -0.9384 - val_acc: 0.6048 - val_mae: 0.1746 - val_msle: 0.0114 - val_mse: 0.0486
Epoch 342/500
21486/21486 [==============================] - 1s 26us/step - loss: -0.9390 - acc: 0.5581 - mae: 0.1745 - msle: 0.0113 - mse: 0.0484 - val_loss: -0.9394 - val_acc: 0.6100 - val_mae: 0.1706 - val_msle: 0.0108 - val_mse: 0.0464
Epoch 343/500
21486/21486 [==============================] - 1s 31us/step - loss: -0.9388 - acc: 0.5596 - mae: 0.1728 - msle: 0.0111 - mse: 0.0474 - val_loss: -0.9383 - val_acc: 0.6041 - val_mae: 0.1684 - val_msle: 0.0107 - val_mse: 0.0453
Epoch 344/500
21486/21486 [===========================

21486/21486 [==============================] - 1s 29us/step - loss: -0.9368 - acc: 0.5471 - mae: 0.1791 - msle: 0.0118 - mse: 0.0511 - val_loss: -0.9370 - val_acc: 0.5951 - val_mae: 0.1783 - val_msle: 0.0118 - val_mse: 0.0508
Epoch 375/500
21486/21486 [==============================] - 1s 28us/step - loss: -0.9372 - acc: 0.5494 - mae: 0.1800 - msle: 0.0119 - mse: 0.0515 - val_loss: -0.9378 - val_acc: 0.6046 - val_mae: 0.1758 - val_msle: 0.0115 - val_mse: 0.0496
Epoch 376/500
21486/21486 [==============================] - 1s 26us/step - loss: -0.9373 - acc: 0.5514 - mae: 0.1781 - msle: 0.0117 - mse: 0.0505 - val_loss: -0.9379 - val_acc: 0.6050 - val_mae: 0.1749 - val_msle: 0.0113 - val_mse: 0.0488
Epoch 377/500
21486/21486 [==============================] - 1s 31us/step - loss: -0.9376 - acc: 0.5539 - mae: 0.1759 - msle: 0.0114 - mse: 0.0492 - val_loss: -0.9381 - val_acc: 0.6074 - val_mae: 0.1743 - val_msle: 0.0113 - val_mse: 0.0486
Epoch 378/500
21486/21486 [===========================

21486/21486 [==============================] - 1s 31us/step - loss: -0.9389 - acc: 0.5620 - mae: 0.1741 - msle: 0.0112 - mse: 0.0481 - val_loss: -0.9372 - val_acc: 0.6085 - val_mae: 0.1722 - val_msle: 0.0110 - val_mse: 0.0471
Epoch 409/500
21486/21486 [==============================] - 1s 28us/step - loss: -0.9386 - acc: 0.5612 - mae: 0.1755 - msle: 0.0114 - mse: 0.0489 - val_loss: -0.9388 - val_acc: 0.6143 - val_mae: 0.1744 - val_msle: 0.0112 - val_mse: 0.0483
Epoch 410/500
21486/21486 [==============================] - 1s 30us/step - loss: -0.9393 - acc: 0.5598 - mae: 0.1755 - msle: 0.0114 - mse: 0.0489 - val_loss: -0.9391 - val_acc: 0.6098 - val_mae: 0.1744 - val_msle: 0.0113 - val_mse: 0.0483
Epoch 411/500
21486/21486 [==============================] - 1s 29us/step - loss: -0.9385 - acc: 0.5553 - mae: 0.1748 - msle: 0.0113 - mse: 0.0485 - val_loss: -0.9386 - val_acc: 0.6076 - val_mae: 0.1737 - val_msle: 0.0112 - val_mse: 0.0480
Epoch 412/500
21486/21486 [===========================

21486/21486 [==============================] - 1s 33us/step - loss: -0.9375 - acc: 0.5507 - mae: 0.1790 - msle: 0.0118 - mse: 0.0509 - val_loss: -0.9378 - val_acc: 0.6141 - val_mae: 0.1755 - val_msle: 0.0114 - val_mse: 0.0489
Epoch 443/500
21486/21486 [==============================] - 1s 40us/step - loss: -0.9380 - acc: 0.5554 - mae: 0.1769 - msle: 0.0116 - mse: 0.0497 - val_loss: -0.9390 - val_acc: 0.6089 - val_mae: 0.1773 - val_msle: 0.0116 - val_mse: 0.0499
Epoch 444/500
21486/21486 [==============================] - 1s 34us/step - loss: -0.9383 - acc: 0.5535 - mae: 0.1799 - msle: 0.0119 - mse: 0.0513 - val_loss: -0.9382 - val_acc: 0.6009 - val_mae: 0.1808 - val_msle: 0.0120 - val_mse: 0.0517
Epoch 445/500
21486/21486 [==============================] - 1s 32us/step - loss: -0.9373 - acc: 0.5511 - mae: 0.1803 - msle: 0.0119 - mse: 0.0516 - val_loss: -0.9367 - val_acc: 0.5957 - val_mae: 0.1771 - val_msle: 0.0116 - val_mse: 0.0497
Epoch 446/500
21486/21486 [===========================

21486/21486 [==============================] - 1s 30us/step - loss: -0.9383 - acc: 0.5541 - mae: 0.1798 - msle: 0.0119 - mse: 0.0511 - val_loss: -0.9382 - val_acc: 0.6070 - val_mae: 0.1817 - val_msle: 0.0120 - val_mse: 0.0520
Epoch 477/500
21486/21486 [==============================] - 1s 29us/step - loss: -0.9381 - acc: 0.5557 - mae: 0.1788 - msle: 0.0118 - mse: 0.0506 - val_loss: -0.9381 - val_acc: 0.6029 - val_mae: 0.1801 - val_msle: 0.0120 - val_mse: 0.0512
Epoch 478/500
21486/21486 [==============================] - 1s 31us/step - loss: -0.9385 - acc: 0.5594 - mae: 0.1789 - msle: 0.0118 - mse: 0.0508 - val_loss: -0.9386 - val_acc: 0.6072 - val_mae: 0.1782 - val_msle: 0.0117 - val_mse: 0.0504
Epoch 479/500
21486/21486 [==============================] - 1s 30us/step - loss: -0.9386 - acc: 0.5625 - mae: 0.1787 - msle: 0.0117 - mse: 0.0507 - val_loss: -0.9382 - val_acc: 0.6044 - val_mae: 0.1800 - val_msle: 0.0118 - val_mse: 0.0514
Epoch 480/500
21486/21486 [===========================

In [11]:
og = layer_extract('encoded', model, x_train)
make_graph(np.array(og), n3['text'], n3['entry_id'], name = en_name + activ_h + '.html', PCA = False)

In [12]:
sub_total = total_df[total_df['entry_id'].isin(list(n5['entry_id']))][['diarynum', 'entry_year', 'entry_id']]
sub_total = sub_total.reset_index(drop = True)
sub_total2 = pd.concat([n5['entry_id'], pd.DataFrame(og)], axis = 1)
sub_total = sub_total.merge(sub_total2, left_on = 'entry_id', right_on = 'entry_id', how = 'inner')
sub_total.to_csv('./102520_gazet.csv')